In [1]:
import numpy as np
import pandas as pd
import json
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

In [2]:
# Load "schools" file.

schools = pd.read_csv("../4_Map/geo_files/schools.csv", encoding = "utf8")

In [3]:
# Load GeoJSON information.

with open("../4_Map/geo_files/geojson_districts") as data_file:
    geojson_districts = json.load(data_file, encoding = "utf8")

In [19]:
sum([i["properties"]["Ninos"] for i in geojson_districts["features"]])

175360.0

In [16]:
[i["properties"]["Plazas"] for i in geojson_districts["features"] if i["properties"]["Distrito"] == "Arganzuela"][0]

3674.0

In [12]:
# Sin la capa de puntos de encima.

app = dash.Dash()


mapbox_access_token = "pk.eyJ1IjoiZ2lsdHJhcG8iLCJhIjoiY2o4eWJyNzY4MXQ1ZDJ3b2JsZHZxb3N0ciJ9.MROnmydnXtfjqjIBtC-P5g"

app.layout = html.Div([
    html.Div(["Schoolarize"    
    ], style = {
        "height": "10%",
        "background-color": "blue"}),
    html.Div([
        dcc.Graph(id = "map", style = {"height": "100%"})
    ],
        style = {"height": "90%"}
    ),
    html.Div([
        dcc.Dropdown(
            id = "district-drop",
            options = [{"label": i["properties"]["Distrito"], "value": i["properties"]["Distrito"]}\
                       for i in geojson_districts["features"]],
            placeholder = "Selecciona tu distrito"
        )
    ], style = {
        "position": "absolute",
        "top": "15%",
        "left": "5%",
        "padding": "0px 0px 0px 0px",
        "width": "300px"}
    ),
    html.Div([
        html.Div(id = "zone", style = {"font-weight": "bold", "color": "#aaaaaf"}),
        html.Div(id = "population", style = {"color": "#aaaaaf"}),
        html.Div(id = "n_schools", style = {"color": "#aaaaaf"}),
        html.Div(id = "n_places", style = {"color": "#aaaaaf"}),
        html.Div(id = "n_population", style = {"color": "#aaaaaf"})
    ], style = {
        "position": "absolute",
        "top": "50%",
        "left": "5%",
        "border-style": "solid",
        "border-width": "1px",
        "border-radius": "10px",
        "border-color": "#d8d8d8",
        "padding": "10px 10px 10px 10px",
        #"width": "280px",
        "background-color": "#ffffff"}
    )
], style = {
    "height": "98vh"})

@app.callback(
    Output("zone", "children"),
    [Input("district-drop", "value")]
)
def update_zone(district):
    if district is not None:
        zone = "Distrito " + district
        return zone
    else:
        return "Ciudad de Madrid"


@app.callback(
    Output("population", "children"),
    [Input("district-drop", "value")]
)
def update_population(district):
    if district is not None:
        nschools = "Población de entre 6 y 11 años: " +\
        '{:,}'.format(int(sum([i["properties"]["Ninos"] for i in geojson_districts["features"]\
                               if i["properties"]["Distrito"] == district][0])))\
        .replace(",", ".")
    else:
        nschools = "Población de entre 6 y 11 años: " +\
        '{:,}'.format(int(sum([i["properties"]["Ninos"] for i in geojson_districts["features"]]))).replace(",", ".")
    return nschools


@app.callback(
    Output("n_schools", "children"),
    [Input("district-drop", "value")]
)
def update_nschools(district):
    if district is not None:
        nschools = "Colegios Públicos: " + str(schools[schools["Distrito"] == district].shape[0])
    else:
        nschools = "Colegios Públicos: "  + str(schools.shape[0])
    return nschools


@app.callback(
    Output("n_places", "children"),
    [Input("district-drop", "value")]
)
def update_nplaces(district):
    if district is not None:
        nplaces = "Plazas (aproximadas): " +\
        '{:,}'.format(int([i["properties"]["Plazas"] for i in geojson_districts["features"]\
                           if i["properties"]["Distrito"] == district][0])).replace(",", ".")
    else:
        nplaces = "Plazas (aproximadas): " +\
        '{:,}'.format(int(sum([i["properties"]["Plazas"] for i in geojson_districts["features"]]))).replace(",", ".")
    return nplaces

    
@app.callback(
    Output("map", "figure"),
    [Input("district-drop", "value")]
)
def update_graph(district):

    if district is not None:
        
        dots = 12
        
        star = 9
        
        square = 8
        
        df = schools[schools["Distrito"] == district]
        
        zoom = [i["properties"]["Zoom"] for i in geojson_districts["features"]\
                if i["properties"]["Distrito"] == district][0]
        
        centerlat = [i["properties"]["Latitud"] for i in geojson_districts["features"]\
                     if i["properties"]["Distrito"] == district][0]
        
        centerlon = [i["properties"]["Longitud"] for i in geojson_districts["features"]\
                     if i["properties"]["Distrito"] == district][0]
        
        district_layer = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"] if i["properties"]["Distrito"] == district],
                    type = geojson_districts["type"]
                ),
                color = [i["properties"]["Color"] for i in geojson_districts["features"]][0],
                opacity = 0.5,
                below = "state-label-lg"
            )
        ]
        
    else:
        
        df = schools
        
        dots = 8
        
        star = 6
        
        square = 7
        
        zoom = 10.2
        
        centerlat = 40.477
        
        centerlon = -3.72
        
        district_layer = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                crs = geojson_districts["crs"],
                features = [i for i in geojson_districts["features"]\
                            if (i["properties"]["NinosxPlaza"] > 1) & (i["properties"]["NinosxPlaza"] < 2)],
                type = geojson_districts["type"]
                ),
                color = "#e0fbfc",
                opacity = 0.5,
                below = "state-label-lg"
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if (i["properties"]["NinosxPlaza"] > 2) & (i["properties"]["NinosxPlaza"] < 3)],
                    type = geojson_districts["type"]
                ),
                color = "#c2dfe3",
                opacity = 0.5,
                below = "state-label-lg"
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if (i["properties"]["NinosxPlaza"] > 3) & (i["properties"]["NinosxPlaza"] < 4)],
                    type = geojson_districts["type"]
                ),
                color = "#9db4c0",
                opacity = 0.5,
                below = "state-label-lg"
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if (i["properties"]["NinosxPlaza"] > 4) & (i["properties"]["NinosxPlaza"] < 5)],
                    type = geojson_districts["type"]
                ),
                color = "#5c6b73",
                opacity = 0.5,
                below = "state-label-lg"
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if (i["properties"]["NinosxPlaza"] > 8) & (i["properties"]["NinosxPlaza"] < 9)],
                    type = geojson_districts["type"]
                ),
                color = "#253237",
                opacity = 0.5,
                below = "state-label-lg"
            )
        ]
    
    return dict(
        data = [
            dict(
                type = "scattermapbox",
                name = "0% de admisiones",
                lat = df[df["Admisiones_p"] == 0]["Latitud"],
                lon = df[df["Admisiones_p"] == 0]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[df["Admisiones_p"] == 0]["Colegio"],
                marker = dict(
                    size = dots,
                    color = "#fe5f55"
                )
            ),
            dict(
                type = "scattermapbox",
                name = "Entre el 1% y el 24%",
                lat = df[(df["Admisiones_p"] > 0) & (df["Admisiones_p"] < 25)]["Latitud"],
                lon = df[(df["Admisiones_p"] > 0) & (df["Admisiones_p"] < 25)]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[(df["Admisiones_p"] > 0) & (df["Admisiones_p"] < 25)]["Colegio"],
                marker = dict(
                    size = dots,
                    color = "#f0b67f"
                )
            ),
            dict(
                type = "scattermapbox",
                name = "Entre el 25% y el 49%",
                lat = df[(df["Admisiones_p"] >= 25) & (df["Admisiones_p"] < 50)]["Latitud"],
                lon = df[(df["Admisiones_p"] >= 25) & (df["Admisiones_p"] < 50)]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[(df["Admisiones_p"] >= 25) & (df["Admisiones_p"] < 50)]["Colegio"],
                marker = dict(
                    size = dots,
                    color = "#d6d1b1"
                )
            ),
            dict(
                type = "scattermapbox",
                name = "Entre el 50% y el 74%",
                lat = df[(df["Admisiones_p"] >= 50) & (df["Admisiones_p"] < 75)]["Latitud"],
                lon = df[(df["Admisiones_p"] >= 50) & (df["Admisiones_p"] < 75)]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[(df["Admisiones_p"] >= 50) & (df["Admisiones_p"] < 75)]["Colegio"],
                marker = dict(
                    size = dots,
                    color = "#c7efcf"
                )
            ),
            dict(
                type = "scattermapbox",
                name = "Entre el 75% y el 99%",
                lat = df[(df["Admisiones_p"] >= 75) & (df["Admisiones_p"] < 100)]["Latitud"],
                lon = df[(df["Admisiones_p"] >= 75) & (df["Admisiones_p"] < 100)]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[(df["Admisiones_p"] >= 75) & (df["Admisiones_p"] < 100)]["Colegio"],
                marker = dict(
                    size = dots,
                    color = "#eef5db"
                )
            ),
            dict(
                type = "scattermapbox",
                name = "100% de admisiones",
                lat = df[df["Admisiones_p"] == 100]["Latitud"],
                lon = df[df["Admisiones_p"] == 100]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[df["Admisiones_p"] == 100]["Colegio"],
                marker = dict(
                    symbol = "star",
                    size = star
                )
            ),
            dict(
                type = "scattermapbox",
                name = "Sin datos",
                lat = df[df["Admisiones_p"] == -1]["Latitud"],
                lon = df[df["Admisiones_p"] == -1]["Longitud"],
                hoverinfo = "text",
                mode = "markers",
                text = df[df["Admisiones_p"] == -1]["Colegio"],
                marker = dict(
                    symbol = "square",
                    size = square
                )
            )
        ],
        layout = dict(
            autosize = True,
            hovermode = "closest",
            margin = dict(l = 0, r = 0, t = 0, b = 0),
            legend = dict(
                xanchor = "right",
                x = 0.95,
                yanchor = "top",
                y = 0.9,
                font = dict(
                    #family = "sans-serif",
                    size = 14,
                    color = "#aaaaaf"
                ),
                bgcolor = "#ffffff",
                bordercolor = "#d8d8d8",
                borderwidth = 1
            ),
            mapbox = dict(
                accesstoken = mapbox_access_token,
                bearing = 0,
                center = dict(
                    lat = centerlat,
                    lon = centerlon
                ),
                zoom = zoom,
                style = "light",
                layers = district_layer
            )
        )
    )

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2017 10:25:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2017 10:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 481, in format
    record.exc_